## Google Colab Usage


In [ ]:
!git clone https://github.com/vanderschaarlab/AutoPrognosis-Multimodal

In [ ]:
%cd AutoPrognosis-Multimodal

## Local Usage


In [ ]:
%cd ../

Verify current working directory

In [ ]:
import os
from pathlib import Path

assert (
    Path(os.getcwd()).name == "AutoPrognosis-Multimodal"
), f"It seems like your are in the wrong directory ({os.getcwd()}), make sure to change to the autoprognosis_m directory."

### Install dependencies


In [ ]:
!pip install -r requirements.txt
!pip install --force-reinstall --no-deps git+https://github.com/u7122029/imgaug.git@numpy-types-fix

## Download and preprocess the data


In [ ]:
!python3 PAD-UFES/preprocess.py --data_dir data --split_dir PAD-UFES/ --resize

## Configure Autoprognosis-m

Note that several of the settings have been set for speed to enable a demonstration. The below settings will __not__ yield optimal performance.

In [ ]:
_globals = {
    "base_experiment_dir": "experiments/fold0/",
    "base_checkpoint_dir": "checkpoints/fold0/",
    "index_column": "img_id",
    "target_column": "diagnostic",
    "class_to_idx": {"ACK": 0, "BCC": 1, "MEL": 2, "NEV": 3, "SCC": 4, "SEK": 5},
    "feature_columns": [
        "age",
        "region_ARM",
        "region_NECK",
        "region_FACE",
        "region_HAND",
        "region_FOREARM",
        "region_CHEST",
        "region_NOSE",
        "region_THIGH",
        "region_SCALP",
        "region_EAR",
        "region_BACK",
        "region_FOOT",
        "region_ABDOMEN",
        "region_LIP",
        "itch_False",
        "itch_True",
        "grew_False",
        "grew_True",
        "hurt_False",
        "hurt_True",
        "changed_False",
        "changed_True",
        "bleed_False",
        "bleed_True",
        "elevation_False",
        "elevation_True",
    ],
    "augmentation": True,
    "fine_tune_lr": 0.0001,
    "gradient_clip_val": 1.0,
    "lr": 1e-6,
    "max_epochs": 10,
    "min_epochs": 5,
    "monitor_metric": "val_loss",
    "monitor_mode": "min",
    "num_workers": 2,
    "patience": 3,
    "seed": 42,
    "warm_up_epochs": 5,
    "batch_size": 16,
    "weighted_loss": True,
}

# Select imaging model from src/models.py: [
#   ResNet18,
#   ResNet34,
#   ResNet50,
#   ResNet101,
#   ResNet152,
#   EfficientNetB0,
#   EfficientNetB1,
#   EfficientNetB2,
#   EfficientNetB3,
#   EfficientNetB4,
#   EfficientNetB5,
#   EfficientNetB6,
#   EfficientNetB7,
#   MobileNetV2,
#   ViTBase,
#   ViTLarge,
#   DinoV2Small,
#   DinoV2Base,
#   DinoV2Large,
#   VitMAEBase,
#   VitMAELarge,
# ]
imaging_config = {"type": "imaging", "model": "ResNet18"}
tabular_config = {
    "type": "tabular",
    "classifier": {
        # Configurations for autoprognosis
        "feature_scaling": ["nop"],
        "feature_selection": ["nop"],
        "classifiers": ["xgboost"],
        "n_folds_cv": 4,
        "timeout": 120,
        "num_study_iter": 1,
        "metric": "aucroc",
        "sample_for_search": True,
        "score_threshold": 0.4,
    },
}

config = {
    "globals": _globals,
    "configurations": [
        # imaging only model
        imaging_config,
        # tabular only model
        tabular_config,
        # joint fusion model
        {"type": "joint_fusion", "model": "ResNet18"},
        # late fusion model
        {
            "type": "late_fusion",
            "imaging": imaging_config,
            "tabular": tabular_config,
        },
        # early fusion model
        {
            "type": "early_fusion",
            "model": "Classifier",
            "imaging": imaging_config,
        },
    ],
}

### Run Autoprognosis-M

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
from src.autoprognosis_m import AutoprognosisM
from src.utils.utils import dict_to_namespace
import pandas as pd

train_csv = "data/folds/fold_0/train.csv"
val_csv = "data/folds/fold_0/val.csv"
test_csv = "data/folds/fold_0/test.csv"

pipeline_config = dict_to_namespace(config)
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)
test_df = pd.read_csv(test_csv)
top_n = 3

autoprognosisM = AutoprognosisM(pipeline_config)
autoprognosisM.run(train_df=train_df, val_df=val_df)
weights = autoprognosisM.fit(df=val_df, top_n=top_n)
ensemble_predictions_df = autoprognosisM.predict(df=test_df, return_probs=True)

### Examine final ensemble

In [ ]:
# Explore components of final ensemble
print("Ensemble components:")
for w, idx in sorted(zip(autoprognosisM.weights, autoprognosisM.top_n_idx), key = lambda x:x[0], reverse=True):
    print(f"\t- {autoprognosisM.pipeline_config.configurations[idx].type}: {w*100:.1f}%")

In [ ]:
# Get performance of final ensemble
from src.utils.metrics import get_metric

gt = pd.get_dummies(test_df[_globals['target_column']],
                    columns=list(sorted(test_df[_globals['target_column']].unique())),
                    dtype=int,
                    ).values.argmax(axis=1)

score = get_metric(preds=ensemble_predictions_df,
                   labels=gt,
                   task='multi-class',
                   metric="Bal. Acc.",
                   )
print(f"Ensemble test performance: {score*100:.1f}%")

### Compare with individual approaches

In [ ]:
from src.imaging.imaging_predict_prob import imaging_predict_prob
from src.late_fusion.late_fusion import late_fusion_predict_prob
from src.early_fusion.early_fusion_predict_prob import early_fusion_predict_prob
from src.joint_fusion.joint_fusion_predict_prob import joint_fusion_predict_prob
from src.tabular.autoprognosis import tabular_predict_prob

In [ ]:
prop_test_dfs = []
scores, config_types = [], []
for model_config in autoprognosisM.pipeline_config.configurations:
    if model_config.type == "imaging":
        # Run imaging prediction
        prop_test_dfs.append(imaging_predict_prob(model_config, df=test_df))
    elif model_config.type == "tabular":
        # Run tabular prediction
        prop_test_dfs.append(tabular_predict_prob(model_config, df=test_df))
    elif model_config.type == "joint_fusion":
        # Run joint fusion prediction
        prop_test_dfs.append(joint_fusion_predict_prob(model_config, df=test_df))
    elif model_config.type == "early_fusion":
        # Run early fusion prediction
        prop_test_dfs.append(early_fusion_predict_prob(model_config, df=test_df))
    elif model_config.type == "late_fusion":
        # Run late fusion prediction
        prop_test_dfs.append(late_fusion_predict_prob(model_config, df=test_df))
    else:
        raise ValueError(f"Unknown model type: {model_config.type}")

    score = get_metric(preds=prop_test_dfs[-1],
                       labels=gt,
                       task='multi-class',
                       metric="Bal. Acc.",
                       )
    scores.append(score)
    config_types.append(model_config.type)

In [ ]:
for c, s in zip(config_types, scores):
    print(f"{c} test performance: \t {s*100:.1f}%")